<a href="https://colab.research.google.com/github/Dats65lol/DESEMPE-O_ECONOMICO/blob/main/PROYECTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:

"""""DEFINITIVO"""""


"""LIBRERIAS"""

import requests  # Para hacer solicitudes a la API del Banco Mundial
import pandas as pd  # Para manejar datos en tablas (DataFrames)
import plotly.graph_objects as go  # Para crear gráficos interactivos

"""CLASES PARA EL API"""

class IndicadorEconomico:
    """Representa un indicador económico como el PIB per cápita de un país en un año."""
    def __init__(self, nombre, pais, valor, year):
        self.nombre = nombre  # Nombre del indicador (ej. PIB per cápita)
        self.pais = pais      # Código del país (ej. 'COL')
        self.valor = valor    # Valor numérico del indicador
        self.year = year      # Año (como string)

class AnalizadorEconomico:
    """Clase para almacenar y graficar indicadores económicos."""
    def __init__(self):
        self.indicadores = []  # Lista vacía para guardar los indicadores

    def agregar_indicador(self, indicador):
        """Agrega un objeto IndicadorEconomico a la lista"""
        self.indicadores.append(indicador)

    def generar_grafico_interactivo(self, paises_seleccionados, start_year, end_year):
        """Genera un gráfico interactivo con los datos disponibles."""
        datos_filtrados = [i for i in self.indicadores if i.pais in paises_seleccionados and start_year <= int(i.year) <= end_year]
        if not datos_filtrados:
            print("No hay datos para los países y años seleccionados.")
            return

        # Organizar los datos por país
        datos_por_pais = {}
        for i in datos_filtrados:
            if i.pais not in datos_por_pais:
                datos_por_pais[i.pais] = {'Año': [], 'PIB per cápita (USD)': []}
            datos_por_pais[i.pais]['Año'].append(int(i.year))
            datos_por_pais[i.pais]['PIB per cápita (USD)'].append(i.valor)

        # Crear gráfico
        fig = go.Figure()
        for pais in paises_seleccionados:
            if pais in datos_por_pais:
                fig.add_trace(go.Scatter(
                    x=datos_por_pais[pais]['Año'],
                    y=datos_por_pais[pais]['PIB per cápita (USD)'],
                    mode='lines+markers',
                    name=pais
                ))

        # Estilo del gráfico
        fig.update_layout(
            title=f"Evolución del PIB per cápita ({', '.join(paises_seleccionados)}, {start_year}-{end_year})",
            xaxis_title="Año",
            yaxis_title="PIB per cápita (USD)",
            showlegend=True
        )

        fig.show()

"""
HACEMOS PRIMERO LAS CLASES YA QUE ES MEJOR Y MAS FACIL, YA
LUEGO HACEMOS LAS FUNCIONES, ADEMÁS REALIZAMOS LA INVESTIGACION EN:
https://datahelpdesk.worldbank.org/knowledgebase/articles/898581
"""

"""TRAEMOS LOS DATOS DEL API PARA PODER HACER EL ANALISIS POR PAIS Y AÑOS DEL PIB PER CAPITA"""

def listar_paises():
    """Descarga y muestra la lista de países disponibles en la API del Banco Mundial."""
    url = "http://api.worldbank.org/v2/country?per_page=500&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1:
            paises = []
            for entry in data[1]:
                if entry['region']['value'] != 'Aggregates':
                    paises.append({'Código': entry['id'], 'Nombre': entry['name']})
            df = pd.DataFrame(paises)
            print("Códigos de países disponibles:")
            print(df.to_string(index=False))
        else:
            print("Formato de respuesta inesperado.")
    else:
        print(f"Error al acceder a la API: Código {response.status_code}")

def descargar_datos_api(paises, start_year, end_year, indicador="NY.GDP.PCAP.CD"):
    """Descarga datos de PIB per cápita desde la API del Banco Mundial."""
    datos = []
    for pais in paises:
        for year in range(start_year, end_year + 1):
            url = f"http://api.worldbank.org/v2/country/{pais}/indicator/{indicador}?date={year}&format=json"
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                if len(data) > 1:
                    for entry in data[1]:
                        value = entry['value']
                        if value is not None:
                            datos.append((pais, value, year))
                else:
                    print(f"Datos incompletos para {pais} en {year}.")
            else:
                print(f"Error al descargar datos para {pais} en {year}: {response.status_code}")
    return datos

"""COMO LO ANTERIOR NECESITA EJERCUCION ES LO QUE VAMOS A REALIZAR PROGRAMANDO UNA FUNCION QUE NOS SIRVA PARA ELLO"""

def main():
    """Ejecuta todo el flujo del programa paso a paso."""
    print("Bienvenido al Analizador de PIB per cápita del Banco Mundial.")
    print("Instrucciones:")
    print("- Ingrese códigos de países de 3 letras (ej: 'COL' para Colombia).")
    print("- Use 'list' para ver códigos disponibles.")
    print("- Escriba 'salir' en cualquier momento para finalizar.")
    print("- El programa generará un gráfico interactivo.\n")

    analizador = AnalizadorEconomico()  # Crear objeto principal

    # Solicitar países
    print("Ingrese códigos de países (separados por coma, ej. COL,USA), 'list' para ver códigos, o 'salir' para terminar:")
    paises_input = input().strip()

    if paises_input.lower() == 'salir':
        print("Saliendo del programa.")
        return
    elif paises_input.lower() == 'list':
        listar_paises()
        return

    paises = [pais.strip().upper() for pais in paises_input.split(",") if len(pais.strip()) == 3]
    if not paises:
        print("No se ingresaron códigos válidos. Usando COL como predeterminado.")
        paises = ["COL"]

    # Solicitar años
    start_year = input("Ingrese el año inicial (o escriba 'salir' para terminar): ").strip()
    if start_year.lower() == 'salir':
        print("Saliendo del programa.")
        return

    end_year = input("Ingrese el año final (o escriba 'salir' para terminar): ").strip()
    if end_year.lower() == 'salir':
        print("Saliendo del programa.")
        return

    # Validar si dejaron vacío
    if start_year == "" or end_year == "":
        print("No escribiste los años. Usando 2020-2022 por defecto.")
        start_year = 2020
        end_year = 2022
    else:
        # Convertir a entero
        start_year = int(start_year)
        end_year = int(end_year)
        if start_year > end_year:
            print("El año inicial no puede ser mayor que el final. Usando 2020-2022 por defecto.")
            start_year = 2020
            end_year = 2022

    # Descargar datos
    datos_api = descargar_datos_api(paises, start_year, end_year)
    if not datos_api:
        print("No se obtuvieron datos válidos. Finalizando programa.")
        return

    # Agregar los datos
    for pais, valor, year in datos_api:
        indicador = IndicadorEconomico("PIB per cápita", pais, valor, year)
        analizador.agregar_indicador(indicador)

    # Generar gráfico
    analizador.generar_grafico_interactivo(paises, start_year, end_year)

    # Finalizar
print("Programa finalizado. Gracias por usar el analizador.")


main()

Programa finalizado. Gracias por usar el analizador.
Bienvenido al Analizador de PIB per cápita del Banco Mundial.
Instrucciones:
- Ingrese códigos de países de 3 letras (ej: 'COL' para Colombia).
- Use 'list' para ver códigos disponibles.
- Escriba 'salir' en cualquier momento para finalizar.
- El programa generará un gráfico interactivo.

Ingrese códigos de países (separados por coma, ej. COL,USA), 'list' para ver códigos, o 'salir' para terminar:
salir
Saliendo del programa.


In [26]:
main()

Bienvenido al Analizador de PIB per cápita del Banco Mundial.
Instrucciones:
- Ingrese códigos de países de 3 letras (ej: 'COL' para Colombia).
- Use 'list' para ver códigos disponibles.
- Escriba 'salir' en cualquier momento para finalizar.
- El programa generará un gráfico directamente en Colab.

Ingrese códigos de países (separados por coma, ej. COL,USA), 'list' para ver códigos, o 'salir' para terminar:
COL,VEN,ECU,PAN
Ingrese el año inicial (o escriba 'salir' para terminar): 2010
Ingrese el año final (o escriba 'salir' para terminar): 2013
